[View in Colaboratory](https://colab.research.google.com/github/joaoflf/ml-paper-implementations/blob/master/resnet_cifar10.ipynb)

### Implementation of ResNet on CIFAR10

In [0]:
#Kill Colab VM (if needed)
#!kill -9 -1

Download and extract CIFAR10 dataset

In [6]:
!pip install git+https://github.com/fbcotter/dataset_loading.git@0.0.3#egg=dataset_loading
!pip install progressbar-latest

import os, urllib, tarfile
import progressbar

pbar = None

def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None

if not os.path.exists('./tmp/data'):
  os.makedirs('tmp/data')
  file_path, _ = urllib.request.urlretrieve(url='https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', filename='tmp/data/cifar10', reporthook=show_progress)
  tarfile.open(name=file_path, mode="r:gz").extractall('tmp/data')


  Running setup.py bdist_wheel for progressbar-latest ... - done
  Stored in directory: /content/.cache/pip/wheels/9a/fd/ed/d692629f9f62127fe2831e7c1e616beb854c969dae7472da01
Successfully built progressbar-latest


100% |########################################################################|


Imports and setup tensor board

In [106]:
!git clone https://github.com/mixuala/colab_utils.git
import colab_utils.tboard
import tensorflow as tf
import numpy as np
from dataset_loading import cifar
from functools import partial
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%M%S')
ROOT = %pwd
root_log_dir = os.path.join(ROOT, 'tf_logs')
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=root_log_dir )

fatal: destination path 'colab_utils' already exists and is not an empty directory.
ngrok installed
status: tensorboard=True, ngrok=False
tensorboard url= http://f803553f.ngrok.io


'http://f803553f.ngrok.io'

Load CIFAR10 Queue

In [60]:
train_queue, test_queue, val_queue = cifar.get_cifar_queues(
    './tmp/data/', cifar10=True, download=True)

Starting processing thread 1 for CIFAR Train QueueStarting processing thread 2 for CIFAR Train Queue

Starting processing thread 1 for CIFAR Test Queue
Starting processing thread 2 for CIFAR Test Queue
Starting processing thread 1 for CIFAR Val Queue
Starting processing thread 2 for CIFAR Val Queue


Layer generators

In [0]:
def create_variable(name, shape, initializer=tf.contrib.layers.xavier_initializer()):
  var =  tf.get_variable(name, shape=shape, initializer=initializer)
  return var

def conv_layer(input_layer, input_channels, output_channels, stride, activation, name):
  weights = create_variable(name + '_w', [3, 3, input_channels, output_channels])
  bias = create_variable(name + '_b', [output_channels])
  
  conv = tf.nn.conv2d(input_layer, weights, stride, padding='SAME') + bias
  conv = tf.layers.batch_normalization(conv)
  
  if activation:
    conv = activation(conv)
  return conv

def res_block(input_layer, input_channels, output_channels):
  
  conv1_stride = [1, 2, 2, 1] if input_channels != output_channels else [1, 1, 1, 1]
    
  conv1 = conv_layer(input_layer, input_channels, output_channels, conv1_stride, tf.nn.relu, 'conv1')
  conv2 = conv_layer(conv1, output_channels, output_channels, [1, 1, 1, 1], None, 'conv2')
  
  if input_channels != output_channels:
    pooled_input = tf.nn.avg_pool(input_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    input_layer = tf.pad(pooled_input, [[0, 0], [0, 0], [0, 0], [input_channels// 2, input_channels // 2]])
  
  addition = tf.add(conv2, input_layer, name= 'addition')
  return tf.nn.relu(addition, name= 'activation')

Create model

In [0]:
tf.reset_default_graph()

learning_rate = 0.001
X = tf.placeholder(tf.float32, [None, 32, 32, 3], 'input')
y = tf.placeholder(tf.int32, [None, 10], 'label')

conv_1= tf.nn.relu(tf.nn.conv2d(X, create_variable('iwc',[3, 3, 3, 16]), [1, 1, 1, 1], padding='SAME')+ create_variable('icb',[16]), name='initial_conv')

with tf.variable_scope('res1_1', reuse=tf.AUTO_REUSE ):
  res_1_1 = res_block(conv_1, 16, 16)
with tf.variable_scope('res1_2', reuse=tf.AUTO_REUSE ):
  res_1_2 = res_block(res_1_1, 16, 16)

with tf.variable_scope('res2_1', reuse=tf.AUTO_REUSE ):
  res_2_1 = res_block(res_1_2, 16, 32)
with tf.variable_scope('res2_2', reuse=tf.AUTO_REUSE ):
  res_2_2 = res_block(res_2_1, 32, 32)
  
with tf.variable_scope('res3_1', reuse=tf.AUTO_REUSE ):
  res_3_1 = res_block(res_2_2, 32, 64)
with tf.variable_scope('res3_2', reuse=tf.AUTO_REUSE ):
  res_3_2 = res_block(res_3_1, 64, 64) 

bn_layer = tf.layers.batch_normalization(res_3_2)
relu_layer = tf.nn.relu(bn_layer)
global_pool = tf.reduce_mean(relu_layer, [1, 2])

fc1_w = create_variable('fc1_w',[64,10])
fc1_b = create_variable('fc1_b',[10])

logits = tf.matmul(global_pool, fc1_w, name='fc1')+ fc1_b

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y, name='cross_entropy_per_example')
loss = tf.reduce_mean(cross_entropy, name='loss')
loss_summary = tf.summary.scalar('Loss', loss)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y ,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

Run training and eval 

In [111]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
logdir = "{}/run-{}/".format(root_log_dir, now)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())


with tf.Session() as sess:
  sess.run(init)

  for i in range(20000):
    X_batch, y_batch = train_queue.get_batch(128)
    logits_output,summary_str, _ =sess.run([logits, loss_summary, optimizer], feed_dict={X: X_batch, y: y_batch})
    file_writer.add_summary(summary_str,i)
    if i % 1000 == 0:
      loss_output, acc = sess.run([loss, accuracy], feed_dict={X: X_batch, y: y_batch})
      print("Iter " + str(i) + ", Minibatch Loss= " + "{:.3f}".format(loss_output) + ", Training Accuracy= " + "{:.3f}".format(acc))
    
  print('Done!')
  X_test, y_test = test_queue.get_batch(128)
  print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: X_test, y: y_test}))


Iter 0, Minibatch Loss= 75.877, Training Accuracy= 0.141
Iter 1000, Minibatch Loss= 1.250, Training Accuracy= 0.531
Iter 2000, Minibatch Loss= 0.819, Training Accuracy= 0.727
Iter 3000, Minibatch Loss= 0.751, Training Accuracy= 0.695
Iter 4000, Minibatch Loss= 0.527, Training Accuracy= 0.797
Iter 5000, Minibatch Loss= 0.520, Training Accuracy= 0.805
Iter 6000, Minibatch Loss= 0.445, Training Accuracy= 0.820
Iter 7000, Minibatch Loss= 0.249, Training Accuracy= 0.914
Iter 8000, Minibatch Loss= 0.261, Training Accuracy= 0.922
Iter 9000, Minibatch Loss= 0.173, Training Accuracy= 0.945
Iter 10000, Minibatch Loss= 0.094, Training Accuracy= 0.984
Iter 11000, Minibatch Loss= 0.129, Training Accuracy= 0.969
Iter 12000, Minibatch Loss= 0.080, Training Accuracy= 0.977
Iter 13000, Minibatch Loss= 0.066, Training Accuracy= 0.984
Iter 14000, Minibatch Loss= 0.064, Training Accuracy= 0.992
Iter 15000, Minibatch Loss= 0.033, Training Accuracy= 0.992
Iter 16000, Minibatch Loss= 0.067, Training Accuracy